In [53]:
import pandas as pd
import pickle
from collections import Counter

import xgboost as xgb
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split

from sklearn import metrics

In [2]:

pd.set_option('display.max_rows', 300)

survey = pd.read_csv('/Users/williamyee/Documents/Data Bootcamp/Classification-Project-DSjobs/Data/kaggle-survey-2017/multi_r_.csv', encoding = 'ISO-8859-1')

list(survey.columns)

survey.JobSatisfaction.value_counts()



survey.head()

survey.EmploymentStatus.unique()

survey.groupby('EmploymentStatus').count()

survey.UniversityImportance.value_counts()

survey.JobSatisfaction.value_counts()



/Users/williamyee/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (32,84,87,88,100,110,117,124,125,128,130,131,165) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


7                          1448
8                          1427
6                           765
9                           677
5                           627
10 - Highly Satisfied       589
3                           358
4                           354
1 - Highly Dissatisfied     167
I prefer not to share       148
2                           117
Name: JobSatisfaction, dtype: int64

In [3]:
conditions = [  survey.JobSatisfaction.isin(['7','8','9','10 - Highly Satisfied']),
                survey.JobSatisfaction.isin(['1 - Highly Dissatisfied','2','3','4','5','6','I prefer not to share'])
             ]
choices = [1,0]
survey['JobSatisfaction'] = np.select(conditions, choices, default=survey.JobSatisfaction)


survey = survey[survey.JobSatisfaction.notna()]


In [4]:
survey.shape

(6677, 229)

In [5]:
percent_missing = survey.isnull().sum()*100/len(survey)
missing_value_df = pd.DataFrame({'columnname':survey.columns, 'percent_missing':percent_missing})
missing_value_df

columnname  \
Unnamed: 0                                                                    Unnamed: 0   
GenderSelect                                                                GenderSelect   
Country                                                                          Country   
Age                                                                                  Age   
EmploymentStatus                                                        EmploymentStatus   
StudentStatus                                                              StudentStatus   
LearningDataScience                                                  LearningDataScience   
CodeWriter                                                                    CodeWriter   
CareerSwitcher                                                            CareerSwitcher   
CurrentJobTitleSelect                                              CurrentJobTitleSelect   
TitleFit                                                                        TitleFit   
CurrentEmployerType                                                  CurrentEmployerType   
MLToolNextYearSelect                                                MLToolNextYearSelect   
MLMethodNextYearSelect                                            MLMethodNextYearSelect   
LanguageRecommendationSelect                                LanguageRecommendationSelect   
PublicDatasetsSelect                                                PublicDatasetsSelect   
LearningPlatformSelect                                            LearningPlatformSelect   
LearningPlatformUsefulnessArxiv                          LearningPlatformUsefulnessArxiv   
LearningPlatformUsefulnessBlogs                          LearningPlatformUsefulnessBlogs   
LearningPlatformUsefulnessCollege                      LearningPlatformUsefulnessCollege   
LearningPlatformUsefulnessCompany                      LearningPlatformUsefulnessCompany   
LearningPlatformUsefulnessConferences              LearningPlatformUsefulnessConferences   
LearningPlatformUsefulnessFriends                      LearningPlatformUsefulnessFriends   
LearningPlatformUsefulnessKaggle                        LearningPlatformUsefulnessKaggle   
LearningPlatformUsefulnessNewsletters              LearningPlatformUsefulnessNewsletters   
LearningPlatformUsefulnessCommunities              LearningPlatformUsefulnessCommunities   
LearningPlatformUsefulnessDocumentation          LearningPlatformUsefulnessDocumentation   
LearningPlatformUsefulnessCourses                      LearningPlatformUsefulnessCourses   
LearningPlatformUsefulnessProjects                    LearningPlatformUsefulnessProjects   
LearningPlatformUsefulnessPodcasts                    LearningPlatformUsefulnessPodcasts   
LearningPlatformUsefulnessSO                                LearningPlatformUsefulnessSO   
LearningPlatformUsefulnessTextbook                    LearningPlatformUsefulnessTextbook   
LearningPlatformUsefulnessTradeBook                  LearningPlatformUsefulnessTradeBook   
LearningPlatformUsefulnessTutoring                    LearningPlatformUsefulnessTutoring   
LearningPlatformUsefulnessYouTube                      LearningPlatformUsefulnessYouTube   
BlogsPodcastsNewslettersSelect                            BlogsPodcastsNewslettersSelect   
LearningDataScienceTime                                          LearningDataScienceTime   
JobSkillImportanceBigData                                      JobSkillImportanceBigData   
JobSkillImportanceDegree                                        JobSkillImportanceDegree   
JobSkillImportanceStats                                          JobSkillImportanceStats   
JobSkillImportanceEnterpriseTools                      JobSkillImportanceEnterpriseTools   
JobSkillImportancePython                                        JobSkillImportancePython   
JobSkillImportanceR                                                  JobSkillImportanceR   
JobSkillImportanceSQL         

In [6]:
percent_missing = survey.isnull().sum()*100/len(survey)
missing_value_df = pd.DataFrame({'columnname':survey.columns, 'percent_missing':percent_missing})
seventy_five_data = missing_value_df[missing_value_df.percent_missing<40]

seventy_five_data

survey_minimal_missing= survey[list(seventy_five_data.columnname)]

survey_minimal_missing.reset_index(inplace=True)
X = survey_minimal_missing.drop(columns = 'JobSatisfaction',axis =1)
y = survey_minimal_missing['JobSatisfaction']
X

index  Unnamed: 0                                       GenderSelect  \
0         0           1  Non-binary, genderqueer, or gender non-conforming   
1         3           4                                               Male   
2         4           5                                               Male   
3         5           6                                               Male   
4         6           7                                               Male   
5         8           9                                             Female   
6         9          10                                               Male   
7        11          12                                               Male   
8        14          15                                               Male   
9        15          16                                               Male   
10       21          22                                               Male   
11       22          23                                               Male   
12       23          24                                               Male   
13       24          25                                               Male   
14       25          26                                               Male   
15       26          27                                               Male   
16       27          28                                               Male   
17       28          29                                               Male   
18       30          31                                               Male   
19       31          32                                               Male   
20       32          33                                               Male   
21       33          34                                               Male   
22       34          35                                               Male   
23       37          38                                               Male   
24       41          42                                               Male   
25       42          43                                               Male   
26       44          45                                               Male   
27       45          46                                               Male   
28       48          49                                               Male   
29       51          52                                               Male   
30       52          53                                               Male   
31       55          56                                               Male   
32       59          60                                               Male   
33       60          61                                               Male   
34       61          62                                               Male   
35       62          63                                               Male   
36       65          66                                               Male   
37       66          67                                             Female   
38       67          68                                               Male   
39       68          69                                               Male   
40       69          70                                             Female   
41       72          73                                               Male   
42       74          75                                               Male   
43       75          76                                               Male   
44       77          78                                               Male   
45       78          79                                               Male   
46       80          81                                               Male   
47       81          82                                               Male   
48       82          83                                               Male   
49       83          84                                               Male   
50       86          87     

In [12]:
X_cat = X.select_dtypes(include='object')
X_cat = pd.DataFrame(X_cat,dtype ='str')
imp_freq = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
X_cat_imputed = imp_freq.fit_transform(X_cat)
X_cat_imputed = pd.DataFrame(data = X_cat_imputed, columns = X_cat.columns)

from collections import defaultdict
d = defaultdict(LabelEncoder)
fit = X_cat_imputed.apply(lambda x: d[x.name].fit_transform(x))
fit

GenderSelect  Country  EmploymentStatus  CodeWriter  \
0                3       52                 0           0   
1                2       50                 2           0   
2                2       46                 0           0   
3                2        4                 0           0   
4                2       50                 0           0   
5                1        1                 0           0   
6                2       39                 0           0   
7                2       17                 0           0   
8                2       46                 0           0   
9                2       50                 2           0   
10               2       50                 1           0   
11               2       50                 0           0   
12               2        8                 1           0   
13               2       50                 0           0   
14               2       39                 0           0   
15               2       27                 0           0   
16               2        7                 0           0   
17               2       13                 2           0   
18               2        9                 0           0   
19               2       35                 0           0   
20               2       49                 0           0   
21               2       39                 0           0   
22               2       50                 0           0   
23               2       44                 0           0   
24               2       50                 0           0   
25               2       36                 0           0   
26               2       31                 0           0   
27               2       26                 1           0   
28               2       50                 0           0   
29               2       23                 0           0   
30               2       28                 0           0   
31               2       50                 0           0   
32               2       50                 0           0   
33               2        5                 2           0   
34               2       19                 1           0   
35               2       40                 0           0   
36               2       17                 0           0   
37               1       17                 2           0   
38               2       12                 0           0   
39               2       12                 0           0   
40               1       13                 1           0   
41               2       43                 0           0   
42               2       49                 2           0   
43               2       50                 0           0   
44               2       21                 2           0   
45               2       40                 0           0   
46               2       50                 0           0   
47               2       13                 0           0   
48               2        7                 0           0   
49               2       39                 0           0   
50               1       50                 0           0   
51               1       50                 0           0   
52               1       50                 0           0   
53               2        5                 0           0   
54               2       16                 0           0   
55               1       50                 0           0   
56               2       50                 0           0   
57               2       50                 2           0   
58               1       50                 1           0   
59               2       50                 0           0   
60               2       17                 0           0   
61               2       43                 0           0   
62               1       50                 0           0   
63               2        4                 0           0   
64               2        4              

In [14]:
X_num = X.select_dtypes(include='float64')
imp_freq = SimpleImputer(missing_values=np.nan, strategy='median')
X_num_imputed = imp_freq.fit_transform(X_num)
X_num_imputed = pd.DataFrame(data= X_num_imputed, columns = X_num.columns)


In [15]:
# for i in range(X_cat_imputed.shape[1]):
#     print(X_cat.iloc[:,i].value_counts())


In [29]:
X_imputed = pd.concat([fit,X_num_imputed],axis =1)


In [30]:
y =pd.Series(y, dtype ='int')

In [36]:
X, X_test, y, y_test = train_test_split(X_imputed, y, test_size=.2, random_state=100)




In [38]:
from sklearn.ensemble import GradientBoostingClassifier

gbm = GradientBoostingClassifier()
gbm.fit(X,y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [39]:
gbm.score(X_test,y_test)

0.68413173652694614

In [46]:
Counter((gbm.predict(X_test)))

Counter({1: 1027, 0: 309})

In [50]:
gbm.feature_importances_

array([ 0.00121128,  0.0369193 ,  0.00125271,  0.        ,  0.00798087,
        0.05015987,  0.03930075,  0.00687596,  0.00662712,  0.00153652,
        0.00228036,  0.02972664,  0.01509375,  0.03981345,  0.00201506,
        0.00183399,  0.00177921,  0.01777367,  0.00027369,  0.00880224,
        0.00624366,  0.01203473,  0.00602549,  0.00016319,  0.0371425 ,
        0.00774283,  0.00568026,  0.00862687,  0.00811901,  0.01878263,
        0.00797601,  0.02211773,  0.00264169,  0.01965715,  0.01942926,
        0.00640988,  0.00833731,  0.02425538,  0.00449767,  0.02190722,
        0.12241032,  0.0132264 ,  0.00427436,  0.00036885,  0.01291444,
        0.01037461,  0.03305377,  0.02073166,  0.00901868,  0.06027544,
        0.04543899,  0.00771638,  0.00873771,  0.04408584,  0.00236044,
        0.01212146,  0.00335302,  0.01076186,  0.01654265,  0.00239476,
        0.00161919,  0.00514868,  0.        ,  0.03402357])

In [51]:
X_imputed.columns

Index(['GenderSelect', 'Country', 'EmploymentStatus', 'CodeWriter',
       'CurrentJobTitleSelect', 'TitleFit', 'CurrentEmployerType',
       'MLToolNextYearSelect', 'MLMethodNextYearSelect',
       'LanguageRecommendationSelect', 'PublicDatasetsSelect',
       'LearningPlatformSelect', 'BlogsPodcastsNewslettersSelect',
       'DataScienceIdentitySelect', 'FormalEducation', 'MajorSelect', 'Tenure',
       'PastJobTitlesSelect', 'FirstTrainingSelect', 'MLSkillsSelect',
       'MLTechniquesSelect', 'ParentsEducation', 'EmployerIndustry',
       'EmployerSize', 'EmployerSizeChange', 'EmployerMLTime',
       'EmployerSearchMethod', 'UniversityImportance', 'JobFunctionSelect',
       'WorkHardwareSelect', 'WorkDataTypeSelect', 'WorkProductionFrequency',
       'WorkDatasetSize', 'WorkAlgorithmsSelect', 'WorkToolsSelect',
       'WorkToolsFrequencyPython', 'WorkToolsFrequencyR', 'WorkMethodsSelect',
       'WorkMethodsFrequencyDataVisualization', 'AlgorithmUnderstandingLevel',
       'WorkCh